## Importing Needed Libraries

<p align="justify">insert spiel</p>

In [46]:
# Part 1: Preprocessing

import torch

from torch.utils.data import DataLoader, Subset

from torchvision import datasets, transforms

# Part 2: Creating DCGAN Generator and Discriminator

import torch.nn as nn

# Part 3: Training a DCGAN for Each Underrepresented Class (Cordana, Healthy, Pestalotiopsis)

import shutil

from pathlib import Path

import torch.optim as optim

from torchvision.utils import save_image

import os

# https://docs.pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
# https://pyimagesearch.com/2021/10/25/training-a-dcgan-in-pytorch/
# https://medium.com/@manoharmanok/implementing-dcgan-in-pytorch-using-the-celeba-dataset-a-comprehensive-guide-660e6e8e29d2


## Part 1: Data Preprocessing

<p align="justify">insert spiel</p>

In [36]:
# constants

RAW_DATA_DIR = "../training_data"
GAN_SIZE = (128, 128)
CNN_SIZE = (224, 224)
BANANA_CLASSES  = ["cordana", "healthy", "pestalotiopsis", "sigatoka"]

<p align="justify">insert spiel</p>

In [37]:
transform_gan = transforms.Compose([
    transforms.Resize(GAN_SIZE),  # Resize for DCGAN
    transforms.ToTensor(),        # Convert to tensor
    transforms.Normalize(
        [0.5, 0.5, 0.5], 
        [0.5, 0.5, 0.5],
    )  # Normalize to [-1, 1] for DCGAN
])

transform_cnn = transforms.Compose([
    transforms.Resize(CNN_SIZE),  # Resize for CNN
    transforms.ToTensor(),        # Convert to tensor
])

<p align="justify">insert spiel</p>

In [38]:
def load_gan_data(batch_size = 32, workers = 2, target_class = None):
    # Load full dataset with GAN transformations
    dataset_gan = datasets.ImageFolder(root = RAW_DATA_DIR, transform = transform_gan)

    if target_class:
        # Get class index from the class name
        class_index = dataset_gan.class_to_idx[target_class]

        # Filter indices where target matches
        indices = [i for i, (_, label) in enumerate(dataset_gan.samples) if label == class_index]

        # Wrap in a Subset
        dataset_gan = Subset(dataset_gan, indices)

    # Create DataLoader for the GAN data
    dataloader_gan = DataLoader(dataset_gan, batch_size = batch_size, shuffle = True, num_workers = workers)

    return dataloader_gan

def load_cnn_data(batch_size = 32, workers = 2):
    # Load dataset with CNN transformations
    dataset_cnn = datasets.ImageFolder(root=RAW_DATA_DIR, transform = transform_cnn)
    
    # Create DataLoader for the CNN data
    dataloader_cnn = DataLoader(dataset_cnn, batch_size=batch_size, shuffle = True, num_workers = workers)

    return dataloader_cnn

<p align="justify">insert spiel</p>

In [39]:
def initialize_weights(model):
    classname = model.__class__.__name__

    if classname.find("Conv") != -1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)

    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(model.weight.data, 1.0, 0.02)
        nn.init.constant_(model.bias.data, 0)

## Part 2: Creating the DCGAN Generator and Discriminator Classes

<p align="justify">insert spiel</p>

In [40]:
# constants

BATCH_SIZE = 128        # Number of images per training batch

INPUT_DIMENSION = 100   # Dimensionality of the generator input

NC = 3                  # Number of channels in the training images

NGF = 64                # Base number of feature maps in the generator

NDF = 64                # Base number of feature maps in Discriminator

EPOCHS = 100            # Number of training epochs

LEARNING_RATE = 0.0002  # Learning rate for both optimizers

BETA1 = 0.5             # Beta1 value for the Adam optimizer to help stabilize DCGAN training

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

NGPU = 1  # Number of GPUs to use (0 means CPU only)


In [41]:
# Generator

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()

        self.ngpu = ngpu

        # Generator network composed of a stack of transposed conv blocks
        self.main = nn.Sequential(
            self._block(INPUT_DIMENSION, NGF * 16, 4, 1, 0, bias = False),  # First layer: latent vector -> feature map
            self._block(NGF * 16, NGF * 8, 4, 2, 1, bias = False),          # Upsample to 8 x 8
            self._block(NGF * 8, NGF * 4, 4, 2, 1, bias = False),           # Upsample to 16 x 16
            self._block(NGF * 4, NGF * 2, 4, 2, 1, bias = False),           # Upsample to 32 x 32
            self._block(NGF * 2, NGF, 4, 2, 1, bias = False),               # Upsample to 64 x 64

            nn.ConvTranspose2d(NGF, NC, 4, 2, 1, bias = False),      # Final upsample to 128x128 with RGB output
            nn.Tanh()                                                # Output pixel values in [-1, 1]
        )

    # Helper function to define a generator block:

    # ConvTranspose2d -> BatchNorm2d -> ReLU

    def _block(self, i_channels, o_channels, kernel_size, stride, padding, bias):
        return nn.Sequential(
            nn.ConvTranspose2d(
                i_channels, 
                o_channels, 
                kernel_size, 
                stride, 
                padding, 
                bias = bias),
            nn.BatchNorm2d(o_channels),
            nn.ReLU(True)
        )

    def forward(self, input):
        return self.main(input)


In [42]:
# Discriminator

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()

        self.ngpu = ngpu

        # Discriminator network composed of downsampling conv blocks
        self.main = nn.Sequential(
            self._block(NC, NDF, 4, 2, 1, bias = False, use_batchNorm2D = False), # First block: no BatchNorm
            self._block(NDF,     NDF * 2, 4, 2, 1, bias = False),                 # Downsample to 32 x 32
            self._block(NDF * 2, NDF * 4, 4, 2, 1, bias = False),                 # Downsample to 16 x 16
            self._block(NDF * 4, NDF * 8, 4, 2, 1, bias = False),                 # Downsample to 8 x 8
            self._block(NDF * 8, NDF * 16, 4, 2, 1, bias = False),                # Downsample to 4 x 4

            nn.Conv2d(NDF * 16, 1, 4, 1, 0, bias = False),                        # Final layer: reduce to 1 x 1
            nn.Sigmoid()                                                          # Output probability [0, 1] of real v.s. fake
        )

    # Helper function to define a discriminator block:

    # Conv2d -> (optional) BatchNorm2d -> LeakyReLU

    def _block(self, i_channels, o_channels, kernel_size, stride, padding, bias, use_batchNorm2D = True):
        layers = [nn.Conv2d(
            i_channels, 
            o_channels, 
            kernel_size, 
            stride, 
            padding, 
            bias = bias)]
        
        if use_batchNorm2D:
            layers.append(nn.BatchNorm2d(o_channels))
        
        layers.append(nn.LeakyReLU(0.2, inplace = True))

        return nn.Sequential(*layers)

    def forward(self, input):
        return self.main(input)


## Part 3: Training a DCGAN for Each Underrepresented Class (Cordana, Healthy, Pestalotiopsis)

<p align="justify">insert spiel</p>

In [51]:
GAN_OUTPUT_DIRECTORY_TEST = "../model2/gan_test" # for debugging while training

def prepare_output_directories():

    for cls in BANANA_CLASSES:
        if cls != "sigatoka":
            full_path = Path(GAN_OUTPUT_DIRECTORY_TEST) / cls

            # If the directory exists, remove and recreate it
            if full_path.exists():
                shutil.rmtree(full_path)

            full_path.mkdir(parents = True, exist_ok = True)

prepare_output_directories()

In [52]:
def train_dcgan_per_class(target_class = None):

    if target_class:
        dataloader = load_gan_data(batch_size = BATCH_SIZE, target_class = target_class)
    
    else:
        dataloader = load_gan_data(batch_size = BATCH_SIZE)

    # Initialize Generator and Discriminator
    netG = Generator(ngpu = NGPU).to(DEVICE)
    netD = Discriminator(ngpu = NGPU).to(DEVICE)

    # Initialize weights
    netG.apply(initialize_weights)
    netD.apply(initialize_weights)

    # Handle multi-GPU setup if applicable
    if (DEVICE.type == "cuda") and (NGPU > 1):
        netG = nn.DataParallel(netG, list(range(NGPU)))
        netD = nn.DataParallel(netD, list(range(NGPU)))

    # Loss function
    criterion = nn.BCELoss() # Binary Cross-Entropy Loss

    # Fixed noise for generating sample outputs and tracking progress during training
    fixed_noise = torch.randn(64, INPUT_DIMENSION, 1, 1, device = DEVICE)

    # Optimizers for Generator and Discriminato
    optimizerD = optim.Adam(netD.parameters(), lr = LEARNING_RATE, betas = (BETA1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr = LEARNING_RATE, betas = (BETA1, 0.999))

    # Labels for real and fake images
    real_label = 1.0
    fake_label = 0.0

    # Actual training
    for epoch in range(EPOCHS):
        for i, (real_images, _) in enumerate(dataloader): # Iterate through batches in the dataset
            # 1. Update Discriminator: 
            #    maximize log(D(x)) + log(1 - D(G(z)))

            # 1.A. Train Discriminator on real images
            netD.zero_grad()

            # Format real batch
            real_images = real_images.to(DEVICE)

            size = real_images.size(0)

            label = torch.full((size,), real_label, dtype = torch.float, device = DEVICE)

            # Forward pass real images through Discriminator
            output = netD(real_images).view(-1)

            # Calculate Discriminator loss for real images
            errD_real = criterion(output, label)

            # Backpropagate error for real images
            errD_real.backward()

            # Mean output for real images
            D_x = output.mean().item()

            # 1.B. Train Discriminator on batch of all fake images

            # Generate batch of latent vectors
            noise = torch.randn(size, INPUT_DIMENSION, 1, 1, device = DEVICE)

            # Generate fake images with Generator
            fake = netG(noise)

            # Classify fake images with Discriminator
            label.fill_(fake_label)

            # Forward pass fake images through Discriminator
            output = netD(fake.detach()).view(-1)

            # Calculate Discriminator loss for fake images
            errD_fake = criterion(output, label)

            # Backpropagate error for fake images
            errD_fake.backward()

            # Mean output for fake images
            D_G_z1 = output.mean().item()

            # Compute total Discriminator error = real error + fake error
            errD = errD_real + errD_fake

            # Finally update Discriminator
            optimizerD.step()

            # 2. Update Generator: 
            #    maximize log(D(G(z)))

            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for Generator cost

            # Pass fake images through Discriminator
            output = netD(fake).view(-1)

            # Calculate Generator loss based on Discriminator's output
            errG = criterion(output, label)

            # Backpropagate error for Generator
            errG.backward()

            # Mean output for fake images after Generator update
            D_G_z2 = output.mean().item()

            # Finally update Generator
            optimizerG.step()

        # Debugging: Print losses and monitor training progress

            if i % 50 == 0:
                print(
                  f"Epoch [{epoch}/{EPOCHS}] Batch {i}/{len(dataloader)} \
                    Loss D: {errD.item():.4f}, loss G: {errG.item():.4f} \
                    D(x): {D_x:.4f}, \
                    D(G(z))_real: {D_G_z1:.4f}, D(G(z))_fake: {D_G_z2:.4f}"
                )

            if epoch % 10 == 0:
                fake_images = netG(fixed_noise).detach()

                if target_class:
                    path = f"{GAN_OUTPUT_DIRECTORY_TEST}/{target_class}"
                
                else:
                    path = GAN_OUTPUT_DIRECTORY_TEST

                save_image(
                    fake_images,
                    os.path.join(path, f"sample_epoch_{epoch}.png"),
                    normalize=True
                )

    return netG, netD


In [ ]:
# Train DCGAN for "cordana" class
trained_generator_cordana, _ = train_dcgan_per_class(target_class = "cordana")

Epoch [0/100] Batch 0/2                     Loss D: 2.1565, loss G: 19.9218                     D(x): 0.6356,                     D(G(z))_real: 0.6733, D(G(z))_fake: 0.0000
Epoch [1/100] Batch 0/2                     Loss D: 4.8930, loss G: 12.5076                     D(x): 0.9932,                     D(G(z))_real: 0.9767, D(G(z))_fake: 0.0000
Epoch [2/100] Batch 0/2                     Loss D: 0.4717, loss G: 12.0453                     D(x): 0.8672,                     D(G(z))_real: 0.1865, D(G(z))_fake: 0.0000


In [ ]:
# Train DCGAN for "healthy" class
# trained_generator_healthy, _ = train_dcgan_per_class(target_class = "healthy")

In [ ]:
# Train DCGAN for "pestalotiopsis" class
# trained_generator_pestalotiopsis, _ = train_dcgan_per_class(target_class = "pestalotiopsis")